In [94]:
import pandas as pd
import re

In [2]:
!git clone https://github.com/DV-11/SpanishDialectDiscrimination.git

Cloning into 'SpanishDialectDiscrimination'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 17 (delta 1), reused 4 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (17/17), 10.93 MiB | 17.06 MiB/s, done.
Resolving deltas: 100% (1/1), done.


# Loading the data

In [3]:
preseea_df = pd.read_csv('/content/SpanishDialectDiscrimination/Data/PRESEEA.csv')

In [22]:
preseea_es = preseea_df[preseea_df['country'] == 'España'].reset_index()['text']
preseea_mx = preseea_df[preseea_df['country'] == 'México'].reset_index()['text']

In [23]:
print("Numer of PS dialogues: ", len(preseea_es))
print("Numer of MS dialogues: ", len(preseea_mx))

Numer of PS dialogues:  233
Numer of MS dialogues:  90


# Parsing dialogue

In [84]:
dialogue = preseea_es[0]

In [85]:
split_str = dialogue.split('</Trans>')
info = split_str[0]
transcript = split_str[1]

In [149]:
all = re.findall(r'(E:|I:)(.*?)(?=(?:E:|I:)|$)', transcript)

dialogue_lines = []
current_line = {'E_line': None, 'I_line': None}

for i, j in all:
    stripped_line = j.strip().removesuffix('/').strip()

    if i == 'E:':
        if current_line['I_line'] is not None:
            dialogue_lines.append(current_line)
            current_line = {'E_line': None, 'I_line': None}
        current_line['E_line'] = stripped_line
    elif i == 'I:':
        current_line['I_line'] = stripped_line

    if current_line['E_line'] is not None and current_line['I_line'] is not None:
        dialogue_lines.append(current_line)
        current_line = {'E_line': None, 'I_line': None}

if current_line['E_line'] is not None or current_line['I_line'] is not None:
    dialogue_lines.append(current_line)

In [162]:
dialogue_df = pd.DataFrame(dialogue_lines)
dialogue_df['Line_number'] = dialogue_df.index + 1
dialogue_df = dialogue_df[['Line_number', 'E_line', 'I_line']]

dialogue_df.head()

,Line_number,E_line,I_line
0,1,"<tiempo = ""00:04""/> bueno estamos con P M / de...",siempre íbamos al tú
1,2,<simultáneo> siempre </simultáneo>,<simultáneo> eso está </simultáneo> claro
2,3,hh,siempre al tú / siempre había excepciones que ...
3,4,<simultáneo> ¿se man </simultáneo> tiene ese <...,se mantiene el trato de <vacilación/> / de<ala...
4,5,¿y el <vacilación/> eeh a los clientes<alargam...,normalmente<alargamiento/> los <vacilación/> /...


Clean up unnecessary tags

In [163]:
dialogue_df['E_line'] = dialogue_df['E_line'].str.replace('<alargamiento/>','')
dialogue_df['E_line'] = dialogue_df['E_line'].str.replace('<vacilación/>','')
dialogue_df['E_line'] = dialogue_df['E_line'].str.replace('</simultáneo>','')
dialogue_df['E_line'] = dialogue_df['E_line'].str.replace('<simultáneo>','')

In [164]:
dialogue_df['I_line'] = dialogue_df['I_line'].str.replace('<alargamiento/>','')
dialogue_df['I_line'] = dialogue_df['I_line'].str.replace('<vacilación/>','')
dialogue_df['I_line'] = dialogue_df['I_line'].str.replace('</simultáneo>','')
dialogue_df['I_line'] = dialogue_df['I_line'].str.replace('<simultáneo>','')

# Dialogue Info

In [165]:
tags = [i for i in info.split(' ') if '=' in i]

all_info = {}

for i in tags:
  split_tag = i.split('=')
  k = split_tag[0]
  v = split_tag[1]

  if not re.findall('"([^"]*)"',split_tag[1]):
    all_info[split_tag[0]] = None
  else:
    all_info[split_tag[0]] = re.findall('"([^"]*)"',split_tag[1])[0]

important_keys = ['clave_texto', 'pais', 'fecha_grab', 'fecha_trans']

dialogue_info = {key: all_info[key] for key in important_keys}

In [166]:
dialogue_info

{'clave_texto': 'ALCA_H11_037',
 'pais': 'España',
 'fecha_grab': '1998-06-25',
 'fecha_trans': '2011-03-23'}

# Readable transcript

In [167]:
for i in dialogue_df.iterrows():
  print(i[1].Line_number, 'E:', i[1].E_line)
  print(i[1].Line_number, 'I:', i[1].I_line)
  print('')

1 E: <tiempo = "00:04"/> bueno estamos con P M / de Alcalá de Henares el veinticinco de junio / del noventa y ocho // bueno P como ya nos  / ya hicimos la entrevista vamos a repetirla / eeh te preguntaba entonces / la otra vez / que normalmente tú cómo tratas a la gente ¿de  de tú o de usted?
1 I: siempre íbamos al tú

2 E:  siempre 
2 I:  eso está  claro

3 E: hh
3 I: siempre al tú / siempre había excepciones que es lo que comentábamos pues eeh gente / que de apariencia / te da para  no te da la <silencio/> eeh por decirlo de alguna forma / la confianza de  / de tratarla de tú entonces ibas al usted pero normalmente de  o a la gente mayor que era lo que decíamos del pueblo / gente mayor que de siempre lo  la familia los padres los abuelos la habían tratado / de usted entonces / siempre te queda eso pero si no / siempre  siempre vas al tú / al  tú / se man  

4 E:  ¿se man  tiene ese  ese trato?
4 I: se mantiene el trato de  / de / pues eso / de toda la vida de conocerla como la tía ta